In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.2.0


In [3]:
import tensorflow_datasets as tfds

In [4]:
imdb, info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTLV60Z/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTLV60Z/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTLV60Z/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
import numpy as np

In [6]:
train_data, test_data = imdb['train'], imdb['test']

In [7]:
train_data

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [8]:
training_sequences=[]
training_labels=[]

In [9]:
testing_sequences=[]
testing_labels=[]

In [10]:
for s,l in train_data:
  training_sequences.append(str(s.numpy()))
  training_labels.append(l.numpy())

In [11]:
for s,l in test_data:
  testing_sequences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [12]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [13]:
vocab_size =10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import  pad_sequences

In [17]:
tokenizer= Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sequences)
word_index=tokenizer.word_index
sequences= tokenizer.texts_to_sequences(training_sequences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [18]:
testing_sequences=tokenizer.texts_to_sequences(testing_sequences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [19]:
testing_padded

array([[  11, 1450, 1528, ...,   60,   48,  214],
       [  26, 5473,    3, ...,  971,    1,    1],
       [  21,    4,  615, ..., 1181, 1023,   47],
       ...,
       [   6,  107,   87, ...,   64,  522,  555],
       [   0,    0,    0, ...,   54,   74,  220],
       [   0,    0,    0, ...,  458, 8360, 2149]], dtype=int32)

In [22]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')
])

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [24]:
num_epochs=10
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5057 - accuracy: 0.7279 - val_loss: 0.3447 - val_accuracy: 0.8476
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2518 - accuracy: 0.8994 - val_loss: 0.3690 - val_accuracy: 0.8362
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1108 - accuracy: 0.9692 - val_loss: 0.4335 - val_accuracy: 0.8285
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0302 - accuracy: 0.9955 - val_loss: 0.5288 - val_accuracy: 0.8219
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0072 - accuracy: 0.9994 - val_loss: 0.5943 - val_accuracy: 0.8231
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6426 - val_accuracy: 0.8267
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6996 - val_accuracy: 0.8246
Epoch 